<a href="https://colab.research.google.com/github/Berenger12/Berenger12/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importation des bibliotheque necessaire
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re #Ce module fournit des opérations de correspondance d'expressions régulières similaires à celles trouvées dans Perl.
import torch
import torch.nn as nn

import torchtext
torchtext.disable_torchtext_deprecation_warning()



In [4]:
#verification des fichier existants dans mon directory
os.listdir("/content/sample_data/")

['anscombe.json',
 'README.md',
 'archive.zip',
 'mnist_train_small.csv',
 'california_housing_train.csv',
 'california_housing_test.csv',
 'mnist_test.csv']

In [ ]:
# Lecture des données et affichage des premieres ligne du dataset
df=pd.read_csv("/content/sample_data/archive.zip")
df.head(20)

,Unnamed: 0,EWE,ENGLISH
0,0,Ne nyɔnu aɖe le evi dzim eye wo le kukum nɛ la...,﻿If a woman often loss his baby after he is bo...
1,1,Ŋkɔ sia nye na ŋkɔ si ke ame bubu tsɔna na ɖev...,"This name comes from another person, which mea..."
2,2,Ame si hɔ ɖevi la ƒlela tsona ƒome bubu me alo...,This person must not be part of the whole fami...
3,3,Kɔnua wo yina ale: evinɔ si ga dzi ɖevi bubu a...,The ceremony is done as follow: the family of ...
4,4,Ne ame aɖe vayina to afimagodzi he kɔ ɖevia la...,When somebody passes through the road and find...
5,5,"Emegbe la ɖevila ƒe pomea ɖona to, he tɔna tek...","After then, the family simply waiting for info..."
6,6,Esiao kãtã vayina le ga ƒoƒo aɖeo me.\n,All this happen in some hours.
7,7,Ɖevi yeye la ƒe ƒomea nana mɔnu kpɔkpɔ amesi f...,The family gives an opportunity to the person ...
8,8,"Le ɣemayi mela ame si fɔ ɖevila, nana ŋkɔ bubu...",At this time this person gives a name he wants...
9,9,"Evi fɔla si trozu ɖevila ƒe tɔ evelia la, ateŋ...",This person who becomes the second father of t...


In [ ]:
# comptage du nombre de ligne et de colonne dans le dataset
df.shape

(28614, 3)

In [ ]:
#information sur les contenu du dataset
df.info

<bound method DataFrame.info of        Unnamed: 0                                                EWE  \
0               0  Ne nyɔnu aɖe le evi dzim eye wo le kukum nɛ la...   
1               1  Ŋkɔ sia nye na ŋkɔ si ke ame bubu tsɔna na ɖev...   
2               2  Ame si hɔ ɖevi la ƒlela tsona ƒome bubu me alo...   
3               3  Kɔnua wo yina ale: evinɔ si ga dzi ɖevi bubu a...   
4               4  Ne ame aɖe vayina to afimagodzi he kɔ ɖevia la...   
...           ...                                                ...   
28609       28609  18 Eye ne Egiptetɔwo meva o la, ekema tsi aɖek...   
28610       28610  19 Esiae anye Egipte kple dukɔ siwo katã mava ...   
28611       28611  20 “Gbe ma gbe la, woaŋlɔ ɖe sɔwo ƒe nyawowoew...   
28612       28612  21 Nuɖaze ɖe sia ɖe si le Yerusalem kple Yuda ...   
28613       28613  Kanaantɔ aɖeke maganɔ aʋakɔwo ƒe Yehowa ƒe aƒe...   

                                                 ENGLISH  
0      ﻿If a woman often loss his baby after he is bo...  
1      This name comes from another person, which mea...  
2      This person must not be part of the whole fami...  
3      The ceremony is done as follow: the family of ...  
4      When somebody passes through the road and find...  
...                                                  ...  
28609  18 and if the clan of Egypt does not come up a...  
28610  19 This will be the penalty for Egypt and the ...  
28611  20 On that day the ponies will have on their j...  
28612  21 and every pot in Jerusalem and in Judah wil...  
28613  And there will no longer be a trader in Jehova...  

[28614 rows x 3 columns]>

In [ ]:
# la description de mes donnees
df.describe()

,Unnamed: 0
count,28614.000000
mean,14306.500000
std,8260.294638
min,0.000000
25%,7153.250000
50%,14306.500000
75%,21459.750000
max,28613.000000


In [ ]:
#verification pour voir si le dataset comporte des valeurs manquantes
df.isnull().sum().any()

False

In [ ]:
# Nettoyage des données
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

df['EWE'] = df['EWE'].apply(clean_text)
df['ENGLISH'] = df['ENGLISH'].apply(clean_text)


In [ ]:
df.head(20)

,Unnamed: 0,EWE,ENGLISH
0,0,ne nynu ae le evi dzim eye wo le kukum n la o ...,if a woman often loss his baby after he is bor...
1,1,k sia nye na k si ke ame bubu tsna na evia si ...,this name comes from another person which mean...
2,2,ame si h evi la lela tsona ome bubu me alo duk...,this person must not be part of the whole fami...
3,3,knua wo yina ale evin si ga dzi evi bubu alo e...,the ceremony is done as follow the family of t...
4,4,ne ame ae vayina to afimagodzi he k evia la le...,when somebody passes through the road and find...
5,5,emegbe la evila e pomea ona to he tna tekp ekp...,after then the family simply waiting for infor...
6,6,esiao kt vayina le ga oo aeo me\n,all this happen in some hours
7,7,evi yeye la e omea nana mnu kpkp amesi f evi l...,the family gives an opportunity to the person ...
8,8,le emayi mela ame si f evila nana k bubu si dz...,at this time this person gives a name he wants...
9,9,evi fla si trozu evila e t evelia la ate ne el...,this person who becomes the second father of t...


In [ ]:
# Division des données
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers)
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):
        src_emb = self.src_embedding(src)
        tgt_emb = self.tgt_embedding(tgt)
        output = self.transformer(src_emb, tgt_emb)
        output = self.fc_out(output)
        return output

In [ ]:
# entrainement du model
def train_model(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        output = model(src, trg[:,:-1])

        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)

        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [ ]:
# evaluation du model

from torchtext.data.metrics import bleu_score
def evaluate_model(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    translations = []
    references = []

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg[:,:-1])
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

            # Collecte des traductions et des références pour calculer le BLEU score
            translations.append(output.argmax(1))
            references.append(trg)

    bleu = bleu_score(translations, references)

    return epoch_loss / len(iterator), bleu



TypeError: Module.eval() missing 1 required positional argument: 'self'